# Extracting title / articles from GDelt


In [37]:
!pip install mwxml
!pip install beautifulsoup4


In [38]:
from google.colab import drive

# drive.mount('/content/drive', force_remount=True)


# FOLDERNAME ='/content/drive/MyDrive/CS329P/Project'

# assert FOLDERNAME is not None, "[!] Enter the foldername."
# %cd /content/drive/MyDrive
# %cd $FOLDERNAME

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Mounted at /content/drive
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-38-1fa6b87c29df>", line 9, in <module>
    get_ipython().magic('cd /content/drive/MyDrive')
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2160, in magic
    return self.run_line_magic(magic_name, magic_arg_s)
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2081, in run_line_magic
    result = fn(*args,**kwargs)
  File "<decorator-gen-84>", line 2, in cd
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/magic.py", line 188, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/magics/osm.py", line 288, in cd
    oldcwd = py3compat.getcwd()
OSError: [Errno 107] Transport endpoint is not connected

Durin

OSError: ignored

In [ ]:

import re
import os
from multiprocessing.pool import ThreadPool
import subprocess
from bs4 import BeautifulSoup
import csv, urllib.request


In [ ]:
def parse_url(url):
    """
    parse the html file from Gdelt

    # For url:

    # To get the data with specific date and time: change 'startdatetime' and 'enddatetime'
    # startdatetime: 2021/09/21/12/21:15
    # enddatetime: 2021/11/02/12/21:15
    # keyword: poverty
    # source country: United states, but we can select the source country with 
    # sourcecountry: __
    # country code is defined here: http://data.gdeltproject.org/api/v2/guides/LOOKUP-COUNTRIES.TXT
    # source language: English, but we can select the source language with 
    # sourcelang:english
    # country code is defined here: http://data.gdeltproject.org/api/v2/guides/LOOKUP-LANGUAGES.TXT

    it only contains the location data in country level(65 diff. countries) and titles, but much faster
    """


    html = urlopen(url).read()
    soup = BeautifulSoup(html, features="html.parser")

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
    return text

In [ ]:
def parse_url_fromcsv(url_csv):
    
    # For csv: 
    # define the format as format=csv, then parse csv to get the each article link
    # url_csv = "https://api.gdeltproject.org/api/v2/doc/doc?query=(%22poverty%22%20OR%20sourcelang:english)&mode=artlist&startdatetime=20210921122115&enddatetime=20211021122115&format=csv"

    response = urllib.request.urlopen(url_csv)
    lines = [l.decode('utf-8') for l in response.readlines()]
    cr = csv.reader(lines)
    output_urls = []
    for row in cr:
        output_urls.append(row[0])

    return output_urls

In [ ]:
url1 = "https://api.gdeltproject.org/api/v2/doc/doc?query=(%22poverty%22%20OR%20sourcecountry:US)&mode=artlist&trans=googtrans&startdatetime=20210921122115&enddatetime=20211021122115"

url = "https://api.gdeltproject.org/api/v2/doc/doc?query=(%22poverty%22%20OR%20sourcelang:english)&mode=artlist&trans=googtrans&startdatetime=20210921122115&enddatetime=20211021122115"

url_csv = "https://api.gdeltproject.org/api/v2/doc/doc?query=(%22poverty%22%20OR%20sourcelang:english)&mode=artlist&startdatetime=20210921122115&enddatetime=20211021122115&format=csv"


In [ ]:
output_html = parse_url(url)

output_url_fromcsv = parse_url_fromcsv(url_csv)

print(output_html)

print(output_url_fromcsv)